In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Basic Transformation 2")
    .master("local[4]")
    .getOrCreate()
)

In [2]:
spark

In [3]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","Female","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [4]:
# Create emp DataFrame
# This won't run yet, lazy evaluation
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [5]:
# Check number of partitions
emp_df.rdd.getNumPartitions()

4

In [6]:
# Show data (ACTION)
emp_df.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [7]:
# Print Schema

emp_df.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [10]:
# Casting Column, another way
# select employee_id, name, age, cast(salary as double) as salary from emp
from pyspark.sql.functions import col, cast

# emp_casted_df = emp_df.selectExpr("employee_id", "name", "age", "cast(salary as double) as salary")
emp_casted_df = emp_df.select("employee_id", "name", "age", col("salary").cast("double"))

In [11]:
emp_casted_df.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: double (nullable = true)



In [12]:
emp_casted_df.show()

+-----------+-------------+---+-------+
|employee_id|         name|age| salary|
+-----------+-------------+---+-------+
|        001|     John Doe| 30|50000.0|
|        002|   Jane Smith| 25|45000.0|
|        003|    Bob Brown| 35|55000.0|
|        004|    Alice Lee| 28|48000.0|
|        005|    Jack Chan| 40|60000.0|
|        006|    Jill Wong| 32|52000.0|
|        007|James Johnson| 42|70000.0|
|        008|     Kate Kim| 29|51000.0|
|        009|      Tom Tan| 33|58000.0|
|        010|     Lisa Lee| 27|47000.0|
|        011|   David Park| 38|65000.0|
|        012|   Susan Chen| 31|54000.0|
|        013|    Brian Kim| 45|75000.0|
|        014|    Emily Lee| 26|46000.0|
|        015|  Michael Lee| 37|63000.0|
|        016|  Kelly Zhang| 30|49000.0|
|        017|  George Wang| 34|57000.0|
|        018|    Nancy Liu| 29|50000.0|
|        019|  Steven Chen| 36|62000.0|
|        020|    Grace Kim| 32|53000.0|
+-----------+-------------+---+-------+



In [13]:
# Adding Columns
# select employee_id, name, age, salary, (salary * 0.2) as tax from emp_casted

emp_taxed_df = emp_casted_df.withColumn("tax", col("salary") * 0.2).withColumn("net", col("salary") - col("tax"))

In [14]:
emp_taxed_df.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: double (nullable = true)
 |-- tax: double (nullable = true)
 |-- net: double (nullable = true)



In [15]:
emp_taxed_df.show()

+-----------+-------------+---+-------+-------+-------+
|employee_id|         name|age| salary|    tax|    net|
+-----------+-------------+---+-------+-------+-------+
|        001|     John Doe| 30|50000.0|10000.0|40000.0|
|        002|   Jane Smith| 25|45000.0| 9000.0|36000.0|
|        003|    Bob Brown| 35|55000.0|11000.0|44000.0|
|        004|    Alice Lee| 28|48000.0| 9600.0|38400.0|
|        005|    Jack Chan| 40|60000.0|12000.0|48000.0|
|        006|    Jill Wong| 32|52000.0|10400.0|41600.0|
|        007|James Johnson| 42|70000.0|14000.0|56000.0|
|        008|     Kate Kim| 29|51000.0|10200.0|40800.0|
|        009|      Tom Tan| 33|58000.0|11600.0|46400.0|
|        010|     Lisa Lee| 27|47000.0| 9400.0|37600.0|
|        011|   David Park| 38|65000.0|13000.0|52000.0|
|        012|   Susan Chen| 31|54000.0|10800.0|43200.0|
|        013|    Brian Kim| 45|75000.0|15000.0|60000.0|
|        014|    Emily Lee| 26|46000.0| 9200.0|36800.0|
|        015|  Michael Lee| 37|63000.0|12600.0|5

In [16]:
# Literals
# select employee_id, name, age, salary, tax, 1 as columnOne, 'two' as columnTwo from emp_taxed
from pyspark.sql.functions import lit

emp_new_cols_df = emp_taxed_df.withColumn("columnOne", lit(1)).withColumn("columnTwo", lit('two'))

In [17]:
emp_new_cols_df.show()

+-----------+-------------+---+-------+-------+-------+---------+---------+
|employee_id|         name|age| salary|    tax|    net|columnOne|columnTwo|
+-----------+-------------+---+-------+-------+-------+---------+---------+
|        001|     John Doe| 30|50000.0|10000.0|40000.0|        1|      two|
|        002|   Jane Smith| 25|45000.0| 9000.0|36000.0|        1|      two|
|        003|    Bob Brown| 35|55000.0|11000.0|44000.0|        1|      two|
|        004|    Alice Lee| 28|48000.0| 9600.0|38400.0|        1|      two|
|        005|    Jack Chan| 40|60000.0|12000.0|48000.0|        1|      two|
|        006|    Jill Wong| 32|52000.0|10400.0|41600.0|        1|      two|
|        007|James Johnson| 42|70000.0|14000.0|56000.0|        1|      two|
|        008|     Kate Kim| 29|51000.0|10200.0|40800.0|        1|      two|
|        009|      Tom Tan| 33|58000.0|11600.0|46400.0|        1|      two|
|        010|     Lisa Lee| 27|47000.0| 9400.0|37600.0|        1|      two|
|        011

In [18]:
# Renaming Columns
# select employee_id as emp_id, name, age, salary, tax, columnOne, columnTwo from emp_new_cols

emp_tax_df = emp_new_cols_df.withColumnRenamed("employee_id", "emp_id")

In [20]:
emp_tax_df.show(3)

+------+----------+---+-------+-------+-------+---------+---------+
|emp_id|      name|age| salary|    tax|    net|columnOne|columnTwo|
+------+----------+---+-------+-------+-------+---------+---------+
|   001|  John Doe| 30|50000.0|10000.0|40000.0|        1|      two|
|   002|Jane Smith| 25|45000.0| 9000.0|36000.0|        1|      two|
|   003| Bob Brown| 35|55000.0|11000.0|44000.0|        1|      two|
+------+----------+---+-------+-------+-------+---------+---------+
only showing top 3 rows



In [23]:
# Remove Column
emp_tax_df_2 = emp_tax_df.drop("columnOne", "columnTwo")

In [24]:
emp_tax_df_2.show(3)

+------+----------+---+-------+-------+-------+
|emp_id|      name|age| salary|    tax|    net|
+------+----------+---+-------+-------+-------+
|   001|  John Doe| 30|50000.0|10000.0|40000.0|
|   002|Jane Smith| 25|45000.0| 9000.0|36000.0|
|   003| Bob Brown| 35|55000.0|11000.0|44000.0|
+------+----------+---+-------+-------+-------+
only showing top 3 rows



In [25]:
# Filter data 
# select employee_id as emp_id, name, age, salary, tax, columnOne from emp_col_dropped where tax > 1000

emp_filterd_df = emp_tax_df_2.where('tax > 1000')

In [26]:
emp_filterd_df.show()

+------+-------------+---+-------+-------+-------+
|emp_id|         name|age| salary|    tax|    net|
+------+-------------+---+-------+-------+-------+
|   001|     John Doe| 30|50000.0|10000.0|40000.0|
|   002|   Jane Smith| 25|45000.0| 9000.0|36000.0|
|   003|    Bob Brown| 35|55000.0|11000.0|44000.0|
|   004|    Alice Lee| 28|48000.0| 9600.0|38400.0|
|   005|    Jack Chan| 40|60000.0|12000.0|48000.0|
|   006|    Jill Wong| 32|52000.0|10400.0|41600.0|
|   007|James Johnson| 42|70000.0|14000.0|56000.0|
|   008|     Kate Kim| 29|51000.0|10200.0|40800.0|
|   009|      Tom Tan| 33|58000.0|11600.0|46400.0|
|   010|     Lisa Lee| 27|47000.0| 9400.0|37600.0|
|   011|   David Park| 38|65000.0|13000.0|52000.0|
|   012|   Susan Chen| 31|54000.0|10800.0|43200.0|
|   013|    Brian Kim| 45|75000.0|15000.0|60000.0|
|   014|    Emily Lee| 26|46000.0| 9200.0|36800.0|
|   015|  Michael Lee| 37|63000.0|12600.0|50400.0|
|   016|  Kelly Zhang| 30|49000.0| 9800.0|39200.0|
|   017|  George Wang| 34|57000

In [27]:
# LIMIT data
# select employee_id as emp_id, name, age, salary, tax from emp_filterd_df limit 5
emp_limit_df = emp_filterd_df.limit(5)

In [28]:
emp_limit_df.show()

+------+----------+---+-------+-------+-------+
|emp_id|      name|age| salary|    tax|    net|
+------+----------+---+-------+-------+-------+
|   001|  John Doe| 30|50000.0|10000.0|40000.0|
|   002|Jane Smith| 25|45000.0| 9000.0|36000.0|
|   003| Bob Brown| 35|55000.0|11000.0|44000.0|
|   004| Alice Lee| 28|48000.0| 9600.0|38400.0|
|   005| Jack Chan| 40|60000.0|12000.0|48000.0|
+------+----------+---+-------+-------+-------+



In [33]:
# Output the data
# Write the data back as CSV (ACTION)

emp_filterd_df.rdd.getNumPartitions()

4

In [34]:
emp_filterd_df.write.format('csv').save('data/2/emp_filterd_df.csv')

![](images/4_partisions_saving.png)

![](images/4_partisions_saving_cat.png)

![](images/4_partisions_saving_cat.png)